In [ ]:
# dependencies
from sqlalchemy import create_engine, inspect, MetaData, select, text, Table, func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Create an engine that can talk to the database
engine = create_engine("sqlite:///Resources/la_crime.db", connect_args={'timeout': 30}, echo=False)
conn = engine.connect()

## Checking Database Data - using SQL

In [ ]:
# Query data from crime table
crime_data = conn.execute("SELECT * FROM crime")

i = 0
for record in crime_data:
    print(record)
    i += 1
    if i == 10:
        break

In [ ]:
# Query data from crime type table
crime_type_data = conn.execute("SELECT * FROM crime_type")

i = 0
for record in crime_type_data:
    print(record)
    i += 1
    if i == 10:
        break

In [ ]:
# Query data from crime table
area_data = conn.execute("SELECT * FROM area")

i = 0
for record in area_data:
    print(record)
    i += 1
    if i == 10:
        break

In [ ]:
# Query data from crime table
premise_data = conn.execute("SELECT * FROM premise")

i = 0
for record in premise_data:
    print(record)
    i += 1
    if i == 10:
        break

## Joining the Tables

In [ ]:
# creating database session
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine=engine, reflect=True)
Base.classes.keys()

In [ ]:
# Collect the names of tables within the database
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Using the inspector to print the column names within the 'crime' table and its types
columns = inspector.get_columns('crime')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Reflect the database tables
metadata =  MetaData(bind=engine)
metadata.reflect()
crime_table = metadata.tables['crime']
area_table = metadata.tables['area']
crime_type_table = metadata.tables['crime_type']
premise_table = metadata.tables['premise']
crime_table

In [ ]:
# Select the crime table
crime_type_tb = engine.execute(select([crime_type_table]))

# Get all the rows
rows = crime_type_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

In [ ]:
# Select the crime type table
crime_tb = engine.execute(select([crime_table]))

# Get all the rows
rows = crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

In [ ]:
# Select the area table
area_tb = engine.execute(select([area_table]))

# Get all the rows
rows = area_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

In [ ]:
# Select the premise table
premise = engine.execute(select([premise_table]))

# Get all the rows
rows = premise.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

In [ ]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")

# Execute the SQL query
la_crime = engine.execute(sql)

In [ ]:
# Print the column names
print(f"Columns: {la_crime.keys()}")

In [ ]:
# Get all the rows
rows = la_crime.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

In [ ]:
# print first 10 rows
i = 0
for row in rows:
    print(row)
    if i == 10:
        break
    i += 1

## Creating the final table for running queries 

In [ ]:
# creating database session
session = Session(engine)
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

In [ ]:
# create the table in the database
metadata.create_all(bind=engine, checkfirst=True)

In [ ]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")
# Execute the SQL query
la_crime = engine.execute(sql)
rows = la_crime.fetchall()
print(len(rows))

# loop over the result set and insert each row into the new table
for row in rows:
    print(row)
    new_row = CrimeJoined(incident_id = row.incident_id, 
                       year = row.year,
                       month = row.month,
                       month_name = row.month_name,
                       victim_age = row.victim_age, 
                       victim_ethnicity = row.victim_ethnicity,
                       victim_gender = row. victim_gender,
                       lat = row.lat,
                       lon = row.lon,
                       area_code = row.area_code,
                       crime_code = row.crime_code, 
                       premise_code= row.premise_code,
                       area_name = row.area_name, 
                       crime = row.crime,
                       premise = row.premise)
    session.add(new_row)

# commit the changes
session.commit()

In [ ]:
inspector.get_table_names()

In [ ]:
# Select the la_crime table
la_crime = engine.execute(select([metadata.tables['la_crime_joined']]))

In [ ]:
# Get all the rows
rows = la_crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

## Data Queries - using SQLAlchamey (copy to flask app to create functions)

use the table named la_crime for all queries (class is CrimeJoined)

In [ ]:
# creating database session
session = Session(engine)
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

In [ ]:
la_crime = engine.execute(select([metadata.tables['la_crime']]))

In [64]:
# Data for overall line chart by year - number of total crimes
query = text("""
    SELECT count('crime'), year, month_name 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Assault'
    GROUP BY month_name
    """)
result = engine.execute(query)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Month": row[2], "Total Crimes": row[0]})
print(crime_dict)

(2470, 2018, 'April')
(2511, 2018, 'August')
(2322, 2018, 'December')
(2035, 2018, 'February')
(2312, 2018, 'January')
(2670, 2018, 'July')
(2572, 2018, 'June')
(2423, 2018, 'March')
(2614, 2018, 'May')
(2262, 2018, 'November')
(2578, 2018, 'October')
(2472, 2018, 'September')
[{'Month': 'April', 'Total Crimes': 2470}, {'Month': 'August', 'Total Crimes': 2511}, {'Month': 'December', 'Total Crimes': 2322}, {'Month': 'February', 'Total Crimes': 2035}, {'Month': 'January', 'Total Crimes': 2312}, {'Month': 'July', 'Total Crimes': 2670}, {'Month': 'June', 'Total Crimes': 2572}, {'Month': 'March', 'Total Crimes': 2423}, {'Month': 'May', 'Total Crimes': 2614}, {'Month': 'November', 'Total Crimes': 2262}, {'Month': 'October', 'Total Crimes': 2578}, {'Month': 'September', 'Total Crimes': 2472}]


In [ ]:
query = text("""
    SELECT count('crime'), year, month_name 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Assault'
    GROUP BY month_name
    ORDER BY CASE month_name
        WHEN 'January' THEN 1
        WHEN 'February' THEN 2
        WHEN 'March' THEN 3
        WHEN 'April' THEN 4
        WHEN 'May' THEN 5
        WHEN 'June' THEN 6
        WHEN 'July' THEN 7
        WHEN 'August' THEN 8
        WHEN 'September' THEN 9
        WHEN 'October' THEN 10
        WHEN 'November' THEN 11
        WHEN 'December' THEN 12
        ELSE 99
    END
""")
result = engine.execute(query)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Month": row[2], "Total Crimes": row[0]})
print(crime_dict)


In [65]:
# Data for overall victim by year and crime-dropped None values
query_age = text("""
    SELECT count(crime), year, Victim_age 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Domestic Violence'
    AND crime IS NOT NULL
    AND Victim_age IS NOT NULL
    GROUP BY Victim_age
""")
result = engine.execute(query_age)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Month": row[2], "Total Crimes": row[0]})
print(crime_dict)


(8, 2018, '1-10')
(1115, 2018, '11-20')
(5726, 2018, '21-30')
(4274, 2018, '31-40')
(2315, 2018, '41-50')
(1158, 2018, '51-60')
(311, 2018, '61-70')
(83, 2018, '71+')
[{'Month': '1-10', 'Total Crimes': 8}, {'Month': '11-20', 'Total Crimes': 1115}, {'Month': '21-30', 'Total Crimes': 5726}, {'Month': '31-40', 'Total Crimes': 4274}, {'Month': '41-50', 'Total Crimes': 2315}, {'Month': '51-60', 'Total Crimes': 1158}, {'Month': '61-70', 'Total Crimes': 311}, {'Month': '71+', 'Total Crimes': 83}]


In [66]:
query_age = text("""
    SELECT count(crime), year, Victim_age 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Domestic Violence'
    AND crime IS NOT NULL
    AND Victim_age IS NOT NULL
    GROUP BY year, Victim_age
""")
result = engine.execute(query_age)
rows = result.fetchall()
crime_dict = []
total_count = sum(row[0] for row in rows)
for row in rows:
    print(row)
    crime_dict.append({"Year": row[1], "Victim Age": row[2], "Total Crimes": row[0], "% of Total": round(row[0]/total_count*100,2)})
print(crime_dict)


(8, 2018, '1-10')
(1115, 2018, '11-20')
(5726, 2018, '21-30')
(4274, 2018, '31-40')
(2315, 2018, '41-50')
(1158, 2018, '51-60')
(311, 2018, '61-70')
(83, 2018, '71+')
[{'Year': 2018, 'Victim Age': '1-10', 'Total Crimes': 8, '% of Total': 0.05}, {'Year': 2018, 'Victim Age': '11-20', 'Total Crimes': 1115, '% of Total': 7.44}, {'Year': 2018, 'Victim Age': '21-30', 'Total Crimes': 5726, '% of Total': 38.2}, {'Year': 2018, 'Victim Age': '31-40', 'Total Crimes': 4274, '% of Total': 28.51}, {'Year': 2018, 'Victim Age': '41-50', 'Total Crimes': 2315, '% of Total': 15.44}, {'Year': 2018, 'Victim Age': '51-60', 'Total Crimes': 1158, '% of Total': 7.73}, {'Year': 2018, 'Victim Age': '61-70', 'Total Crimes': 311, '% of Total': 2.07}, {'Year': 2018, 'Victim Age': '71+', 'Total Crimes': 83, '% of Total': 0.55}]


In [67]:
from sqlalchemy import create_engine

# Create engine and connect to database
engine = create_engine("sqlite:///Resources/la_crime.db")
connection = engine.connect()

# Query to fetch column names
query_cols = "PRAGMA table_info(la_crime)"
result_cols = connection.execute(query_cols)

# Extract column names from result set
cols = [row[1] for row in result_cols]

# Print column names
print(cols)

# Close connection
connection.close()


['id', 'incident_id', 'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_name', 'crime', 'premise']


In [55]:
# 1-crime data by year
query = text("""
    SELECT count('crime'), year, month_name 
    FROM la_crime
    WHERE year = 2018 
    AND crime = 'Assault'
    GROUP BY month_name
    ORDER BY CASE month_name
        WHEN 'January' THEN 1
        WHEN 'February' THEN 2
        WHEN 'March' THEN 3
        WHEN 'April' THEN 4
        WHEN 'May' THEN 5
        WHEN 'June' THEN 6
        WHEN 'July' THEN 7
        WHEN 'August' THEN 8
        WHEN 'September' THEN 9
        WHEN 'October' THEN 10
        WHEN 'November' THEN 11
        WHEN 'December' THEN 12
        ELSE 99
    END
""")
result = engine.execute(query)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Month": row[2], "Total Crimes": row[0]})
print(crime_dict)


(2312, 2018, 'January')
(2035, 2018, 'February')
(2423, 2018, 'March')
(2470, 2018, 'April')
(2614, 2018, 'May')
(2572, 2018, 'June')
(2670, 2018, 'July')
(2511, 2018, 'August')
(2472, 2018, 'September')
(2578, 2018, 'October')
(2262, 2018, 'November')
(2322, 2018, 'December')
[{'Month': 'January', 'Total Crimes': 2312}, {'Month': 'February', 'Total Crimes': 2035}, {'Month': 'March', 'Total Crimes': 2423}, {'Month': 'April', 'Total Crimes': 2470}, {'Month': 'May', 'Total Crimes': 2614}, {'Month': 'June', 'Total Crimes': 2572}, {'Month': 'July', 'Total Crimes': 2670}, {'Month': 'August', 'Total Crimes': 2511}, {'Month': 'September', 'Total Crimes': 2472}, {'Month': 'October', 'Total Crimes': 2578}, {'Month': 'November', 'Total Crimes': 2262}, {'Month': 'December', 'Total Crimes': 2322}]


In [77]:
# 2- Data for victim_ethnicity %
query_age = text("""
  SELECT victim_ethnicity, count(*) AS total_count
    FROM la_crime
    WHERE year = 2018
    AND crime = 'Burglary from Vehicle'
    GROUP BY victim_ethnicity 
    """)
result = engine.execute(query_age)
rows = result.fetchall()
race_dict = []
total_count = 0
for row in rows:
    print(row)
    race_dict.append({"Ethnicity": row[0], "Count": row[1]})
    total_count += row[1]
print(race_dict, total_count)
for d in race_dict:
    d["Total People"] = total_count
    d["Ethnicity %"] = round((d["Count"]/d["Total People"]) * 100)
print(race_dict)







(None, 74)
('Asian', 1486)
('Asian Indian', 4)
('Black', 3606)
('Hispanic', 10371)
('Native American', 7)
('Other', 3491)
('Pacific Islander/Hawaiian', 7)
('White', 9029)
[{'Ethnicity': None, 'Count': 74}, {'Ethnicity': 'Asian', 'Count': 1486}, {'Ethnicity': 'Asian Indian', 'Count': 4}, {'Ethnicity': 'Black', 'Count': 3606}, {'Ethnicity': 'Hispanic', 'Count': 10371}, {'Ethnicity': 'Native American', 'Count': 7}, {'Ethnicity': 'Other', 'Count': 3491}, {'Ethnicity': 'Pacific Islander/Hawaiian', 'Count': 7}, {'Ethnicity': 'White', 'Count': 9029}] 28075
[{'Ethnicity': None, 'Count': 74, 'Total People': 28075, 'Ethnicity %': 0}, {'Ethnicity': 'Asian', 'Count': 1486, 'Total People': 28075, 'Ethnicity %': 5}, {'Ethnicity': 'Asian Indian', 'Count': 4, 'Total People': 28075, 'Ethnicity %': 0}, {'Ethnicity': 'Black', 'Count': 3606, 'Total People': 28075, 'Ethnicity %': 13}, {'Ethnicity': 'Hispanic', 'Count': 10371, 'Total People': 28075, 'Ethnicity %': 37}, {'Ethnicity': 'Native American', 'Coun

In [78]:
# 3- Crme by gender%
query_gender = text("""
    SELECT count(crime), crime, victim_gender 
    FROM la_crime
    WHERE year = 2018 
    AND crime = "Trespassing"
    AND crime IS NOT NULL
    AND victim_gender IS NOT NULL
    GROUP BY crime, victim_gender
""")
result = engine.execute(query_gender)
rows = result.fetchall()

gender_dict = []
for crime in set([row[1] for row in rows]):
    crime_count = sum([row[0] for row in rows if row[1] == crime])
    for gender in set([row[2] for row in rows if row[1] == crime]):
        gender_count = sum([row[0] for row in rows if row[1] == crime and row[2] == gender])
        gender_dict.append({
            #"Crime Type": crime,
            "Victim Gender": gender,
            "Total Crimes": gender_count,
            "% of Total": round((gender_count/crime_count)*100, 2)
        })

print(gender_dict)




[{'Victim Gender': 'Male', 'Total Crimes': 1112, '% of Total': 55.16}, {'Victim Gender': 'Female', 'Total Crimes': 904, '% of Total': 44.84}]


In [53]:
#crime by ethnicity
query_ethnicity = text("""
    SELECT count(crime), crime, victim_ethnicity 
    FROM la_crime
    WHERE year = 2018 
    AND crime IS NOT NULL
    AND victim_ethnicity IS NOT NULL
    GROUP BY crime, victim_ethnicity
""")
result = engine.execute(query_ethnicity)
rows = result.fetchall()
crime_dict = []
for crime in set([row[1] for row in rows]):
    crime_rows = [row for row in rows if row[1] == crime]
    crime_dict_entry = {"Crime Type": crime, "Victim Ethnicity": []}
    total_count = sum(row[0] for row in crime_rows)
    for row in crime_rows:
        crime_dict_entry["Victim Ethnicity"].append({"Ethnicity": row[2], "Total Crimes": row[0], "% of Total": round(row[0]/total_count*100,2)})
    crime_dict.append(crime_dict_entry)
print(crime_dict)




[{'Crime Type': 'Trespassing', 'Victim Ethnicity': [{'Ethnicity': 'Asian', 'Total Crimes': 73, '% of Total': 3.68}, {'Ethnicity': 'Black', 'Total Crimes': 277, '% of Total': 13.95}, {'Ethnicity': 'Hispanic', 'Total Crimes': 559, '% of Total': 28.16}, {'Ethnicity': 'Other', 'Total Crimes': 269, '% of Total': 13.55}, {'Ethnicity': 'White', 'Total Crimes': 807, '% of Total': 40.65}]}, {'Crime Type': 'Violation of Reatraining Order', 'Victim Ethnicity': [{'Ethnicity': 'Asian', 'Total Crimes': 35, '% of Total': 1.46}, {'Ethnicity': 'Black', 'Total Crimes': 443, '% of Total': 18.43}, {'Ethnicity': 'Hispanic', 'Total Crimes': 1323, '% of Total': 55.03}, {'Ethnicity': 'Other', 'Total Crimes': 141, '% of Total': 5.87}, {'Ethnicity': 'White', 'Total Crimes': 462, '% of Total': 19.22}]}, {'Crime Type': 'Bike Theft', 'Victim Ethnicity': [{'Ethnicity': 'Asian', 'Total Crimes': 152, '% of Total': 6.78}, {'Ethnicity': 'Asian Indian', 'Total Crimes': 1, '% of Total': 0.04}, {'Ethnicity': 'Black', 'Tot

In [48]:
query_ethnicity = text("""
    SELECT count(crime), crime, victim_ethnicity 
    FROM la_crime
    WHERE year = 2018 
    AND crime IS NOT NULL
    AND victim_ethnicity IS NOT NULL
    GROUP BY crime, victim_ethnicity
""")
result = engine.execute(query_ethnicity)
rows = result.fetchall()
crime_dict = []
total_count = sum(row[0] for row in rows)
for row in rows:
    print(row)
    crime_dict.append({"Crime Type": row[1], "Victim Ethnicity": row[2], "Total Crimes": row[0], "% of Total": round(row[0]/total_count*100,2)})
print(crime_dict)


(724, 'Assault', 'Asian')
(2, 'Assault', 'Asian Indian')
(7182, 'Assault', 'Black')
(13847, 'Assault', 'Hispanic')
(4, 'Assault', 'Native American')
(2057, 'Assault', 'Other')
(1, 'Assault', 'Pacific Islander/Hawaiian')
(5317, 'Assault', 'White')
(152, 'Bike Theft', 'Asian')
(1, 'Bike Theft', 'Asian Indian')
(215, 'Bike Theft', 'Black')
(473, 'Bike Theft', 'Hispanic')
(19, 'Bike Theft', 'Native American')
(232, 'Bike Theft', 'Other')
(2, 'Bike Theft', 'Pacific Islander/Hawaiian')
(1148, 'Bike Theft', 'White')
(36, 'Brandish Weapon', 'Asian')
(2, 'Brandish Weapon', 'Asian Indian')
(775, 'Brandish Weapon', 'Black')
(1343, 'Brandish Weapon', 'Hispanic')
(171, 'Brandish Weapon', 'Other')
(364, 'Brandish Weapon', 'White')
(1486, 'Burglary from Vehicle', 'Asian')
(4, 'Burglary from Vehicle', 'Asian Indian')
(3606, 'Burglary from Vehicle', 'Black')
(10371, 'Burglary from Vehicle', 'Hispanic')
(7, 'Burglary from Vehicle', 'Native American')
(3491, 'Burglary from Vehicle', 'Other')
(7, 'Burglar

In [51]:
query_location = text("""
    SELECT lat, lon, area_name, crime, premise, count(crime) as total_crimes 
    FROM la_crime 
    WHERE year = 2018 
    AND crime IS NOT NULL 
    AND lat IS NOT NULL 
    AND lon IS NOT NULL 
    AND area_name IS NOT NULL 
    AND premise IS NOT NULL 
    GROUP BY lat, lon, area_name, crime, premise 
""")
result = engine.execute(query_location)
rows = result.fetchall()
crime_dict = []
for row in rows:
    crime_dict.append({
        "Latitude": row[0],
        "Longitude": row[1],
        "Area Name": row[2],
        "Crime Type": row[3],
        "Premise": row[4],
        "Total Crimes": row[5]
    })
print(crime_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
query_location = text("""
    SELECT lat, lon, area_name, crime, premise 
    FROM la_crime
    WHERE year = 2018 
    AND crime IS NOT NULL
    AND lat IS NOT NULL
    AND lon IS NOT NULL
    AND area_name IS NOT NULL
    AND premise IS NOT NULL
    LIMIT 10
""")
result = engine.execute(query_location)
rows = result.fetchall()
crime_dict = []
for row in rows:
    print(row)
    crime_dict.append({"Latitude": row[0], "Longitude": row[1], "Area Name": row[2], "Crime Type": row[3], "Premise": row[4]})
print(crime_dict)


In [52]:
#query to show crime by year, geoloc, area, premis and crime typ
query_location = text("""
    SELECT count(crime), lat, lon, area_name, crime, premise, year
    FROM la_crime
    WHERE year = year 
    AND crime IS NOT NULL
    AND lat IS NOT NULL
    AND lon IS NOT NULL
    AND area_name IS NOT NULL
    AND premise IS NOT NULL
    GROUP BY lat, lon, area_name, crime, premise
    LIMIT 200
""")
result = engine.execute(query_location)
rows = result.fetchall()
crime_dict = []
for row in rows:
    crime_dict.append({"Year": row[6], "Crime Type": row[4], "Premise": row[5], "Area Name": row[3], "Latitude": row[1], "Longitude": row[2], "Total Crimes": row[0]})
print(crime_dict)


[{'Year': 2020, 'Crime Type': 'Assault', 'Premise': 'Business', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 2}, {'Year': 2020, 'Crime Type': 'Assault', 'Premise': 'Freeway/Street', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 4}, {'Year': 2020, 'Crime Type': 'Assault', 'Premise': 'Hotel/Motel', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 1}, {'Year': 2020, 'Crime Type': 'Assault', 'Premise': 'Residence', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 13}, {'Year': 2020, 'Crime Type': 'Assault', 'Premise': 'Sidewalk', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 1}, {'Year': 2021, 'Crime Type': 'Assault', 'Premise': 'Temp Housing', 'Area Name': '77th Street', 'Latitude': 0.0, 'Longitude': 0.0, 'Total Crimes': 1}, {'Year': 2021, 'Crime Type': 'Brandish Weapon', 'Premise': 'Freeway/Street', 'Area Name': '77th Street', 'La

In [ ]:
#query to show crime by year, geoloc, area, premis and crime typ
query_location = text("""
    SELECT count(crime), lat, lon, area_name, crime, premise, year
    FROM la_crime
    WHERE year = 2018
    AND crime = "Assault"
    AND crime IS NOT NULL
    AND lat IS NOT NULL
    AND lon IS NOT NULL
    AND area_name IS NOT NULL
    AND premise IS NOT NULL
    GROUP BY lat, lon, area_name, crime, premise
    LIMIT 200
""")
result = engine.execute(query_location)
rows = result.fetchall()
crime_dict = []
for row in rows:
    crime_dict.append({"Year": row[6], "Crime Type": row[4], "Premise": row[5], "Area Name": row[3], "Latitude": row[1], "Longitude": row[2], "Total Crimes": row[0]})
print(crime_dict)

In [ ]:
# Define the la_crime class , which will have a one-to-one relationship with the Area, premise and crime type tables
class CrimeJoined(Base):
    __tablename__ = 'la_crime'
    id = Column(Integer, primary_key=True, autoincrement=True)
    incident_id = Column(Integer)
    year = Column(Integer)
    month = Column(Integer)
    month_name = Column(String)
    victim_age = Column(String)
    victim_ethnicity = Column(String)
    victim_gender = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    area_code = Column(Integer)
    crime_code = Column(Integer)
    premise_code = Column(Integer)
    area_name = Column(String, nullable=False)
    crime = Column(String, nullable=False)
    premise = Column(String, nullable=False)